# Notebook 02: CUDA Matrix Operations & Softmax

## Building Neural Network Primitives on GPU

Welcome to Notebook 02! Now that you understand CUDA basics, we'll implement essential neural network operations:

1. **Optimized Matrix Multiplication** - Using shared memory and tiling
2. **Stable Softmax** - Critical for attention mechanisms
3. **Memory Coalescing** - Maximizing bandwidth
4. **Performance Optimization** - Profiling and tuning

These operations form the foundation of transformer models!

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Tuple

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Part 1: Matrix Multiplication Optimization

### Understanding Shared Memory

**Memory Hierarchy Speed:**
```
Registers:      ~1 cycle     (fastest, tiny)
Shared Memory:  ~5 cycles    (fast, small - 48KB per block)
Global Memory:  ~400 cycles  (slow, large - GBs)
```

**Tiling Strategy:**
Instead of reading from global memory for every computation:
1. Load tiles into shared memory
2. Compute using fast shared memory
3. Repeat for all tiles

### Naive vs Tiled Matrix Multiplication

![Matrix Multiplication Tiling](https://via.placeholder.com/600x300.png?text=Tiling+Visualization)

In [ ]:
def matmul_naive_gpu(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    """Naive matrix multiplication - each element computed independently."""
    return torch.matmul(A, B)

# PyTorch's optimized version uses tiling internally
# Let's compare with explicit loops to understand the difference

def matmul_explicit_loops(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """Explicit triple-loop implementation (SLOW - for demonstration)."""
    m, k = A.shape
    k2, n = B.shape
    assert k == k2
    
    C = np.zeros((m, n), dtype=np.float32)
    for i in range(m):
        for j in range(n):
            for k_idx in range(k):
                C[i, j] += A[i, k_idx] * B[k_idx, j]
    return C

# Test
size = 256
A = torch.randn(size, size, device=device)
B = torch.randn(size, size, device=device)

# Warmup
_ = torch.matmul(A, B)
torch.cuda.synchronize()

# Benchmark optimized version
start = time.time()
C = torch.matmul(A, B)
torch.cuda.synchronize()
gpu_time = (time.time() - start) * 1000

print(f"Optimized GPU MatMul ({size}×{size}): {gpu_time:.2f} ms")
print(f"\nThroughput: {(2 * size**3) / (gpu_time / 1000) / 1e9:.2f} GFLOPS")

### Visualizing Memory Access Patterns

Understanding memory coalescing is crucial for GPU performance.

In [ ]:
def benchmark_memory_patterns(sizes: list) -> dict:
    """Compare different memory access patterns."""
    results = {'contiguous': [], 'transpose': []}
    
    for size in sizes:
        # Contiguous access (fast)
        A = torch.randn(size, size, device=device)
        torch.cuda.synchronize()
        start = time.time()
        _ = A.sum(dim=1)  # Row-wise (contiguous in memory)
        torch.cuda.synchronize()
        results['contiguous'].append((time.time() - start) * 1000)
        
        # Non-contiguous access (slower)
        torch.cuda.synchronize()
        start = time.time()
        _ = A.sum(dim=0)  # Column-wise (strided access)
        torch.cuda.synchronize()
        results['transpose'].append((time.time() - start) * 1000)
    
    return results

sizes = [512, 1024, 2048, 4096]
mem_results = benchmark_memory_patterns(sizes)

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(sizes, mem_results['contiguous'], 'o-', label='Row-wise (coalesced)', linewidth=2)
plt.plot(sizes, mem_results['transpose'], 's-', label='Column-wise (strided)', linewidth=2)
plt.xlabel('Matrix Size', fontsize=12)
plt.ylabel('Time (ms)', fontsize=12)
plt.title('Memory Access Patterns Impact', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Key Insight: Contiguous memory access is faster!")
print("   GPUs read memory in coalesced chunks (32/128 bytes)")
print("   Arrange data for sequential access when possible")

## Part 2: Softmax - The Foundation of Attention

### Mathematical Definition

For a vector $x = [x_1, x_2, ..., x_n]$:

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}$$

### Numerical Stability Challenge

**Problem:** Large values cause overflow:
- $e^{1000}$ = overflow!
- $e^{-1000}$ = underflow to 0

**Solution:** Subtract max before exponential:

$$\text{softmax}(x_i) = \frac{e^{x_i - \max(x)}}{\sum_{j=1}^{n} e^{x_j - \max(x)}}$$

This is mathematically equivalent but numerically stable!

In [ ]:
def softmax_naive(x: np.ndarray) -> np.ndarray:
    """Naive softmax - UNSTABLE for large values."""
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def softmax_stable(x: np.ndarray) -> np.ndarray:
    """Numerically stable softmax."""
    x_max = np.max(x)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x)

# Demonstrate stability issue
x_small = np.array([1.0, 2.0, 3.0], dtype=np.float32)
x_large = np.array([1000.0, 1001.0, 1002.0], dtype=np.float32)

print("Small values:")
print(f"  Naive:  {softmax_naive(x_small)}")
print(f"  Stable: {softmax_stable(x_small)}")
print(f"  Match: {np.allclose(softmax_naive(x_small), softmax_stable(x_small))}")

print("\nLarge values:")
try:
    naive_result = softmax_naive(x_large)
    print(f"  Naive:  {naive_result}")
    if np.any(np.isnan(naive_result)):
        print("  ⚠️ NaN detected - numerical overflow!")
except:
    print("  ⚠️ Naive version FAILED - overflow!")

stable_result = softmax_stable(x_large)
print(f"  Stable: {stable_result}")
print(f"  Sum:    {np.sum(stable_result):.6f} (should be 1.0)")

### GPU Softmax Implementation

In [ ]:
def softmax_gpu(x: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """Numerically stable GPU softmax."""
    return torch.softmax(x, dim=dim)

def softmax_manual(x: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """Manual stable softmax implementation."""
    x_max = x.max(dim=dim, keepdim=True)[0]
    exp_x = torch.exp(x - x_max)
    return exp_x / exp_x.sum(dim=dim, keepdim=True)

# Test with various shapes
test_cases = [
    ("1D vector", torch.randn(10, device=device)),
    ("2D matrix (row-wise)", torch.randn(100, 50, device=device)),
    ("3D tensor (last dim)", torch.randn(32, 64, 128, device=device))
]

for name, x in test_cases:
    builtin = softmax_gpu(x)
    manual = softmax_manual(x)
    
    print(f"\n{name} - Shape: {x.shape}")
    print(f"  Results match: {torch.allclose(builtin, manual)}")
    print(f"  Sum check: {manual.sum(dim=-1).mean():.6f} (should be 1.0)")
    print(f"  Max value: {manual.max():.6f}")
    print(f"  Min value: {manual.min():.6f}")

### Batch Softmax Performance

In [ ]:
def benchmark_softmax(batch_sizes: list, seq_lengths: list) -> dict:
    """Benchmark softmax across different dimensions."""
    results = {}
    
    for batch in batch_sizes:
        results[batch] = []
        for seq_len in seq_lengths:
            x = torch.randn(batch, seq_len, device=device)
            
            # Warmup
            _ = torch.softmax(x, dim=-1)
            torch.cuda.synchronize()
            
            # Benchmark
            start = time.time()
            for _ in range(100):  # Multiple iterations for accuracy
                _ = torch.softmax(x, dim=-1)
            torch.cuda.synchronize()
            
            avg_time = ((time.time() - start) / 100) * 1000
            results[batch].append(avg_time)
    
    return results

batch_sizes = [1, 32, 128]
seq_lengths = [64, 128, 256, 512, 1024]

print("Benchmarking softmax performance...")
softmax_results = benchmark_softmax(batch_sizes, seq_lengths)

# Visualize
plt.figure(figsize=(12, 6))
for batch in batch_sizes:
    plt.plot(seq_lengths, softmax_results[batch], 'o-', 
             label=f'Batch={batch}', linewidth=2, markersize=8)

plt.xlabel('Sequence Length', fontsize=12)
plt.ylabel('Time per call (ms)', fontsize=12)
plt.title('Softmax Performance vs Sequence Length', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print("\n💡 Observations:")
print("   - Larger batches amortize overhead")
print("   - Time scales roughly linearly with sequence length")
print("   - GPU shines with large batch sizes")

## Part 3: Log-Softmax and Numerical Tricks

### Why Log-Softmax?

In many applications (cross-entropy loss, attention), we need $\log(\text{softmax}(x))$:

**Naive:** $\log(\text{softmax}(x_i)) = \log\left(\frac{e^{x_i}}{\sum_j e^{x_j}}\right)$

**Optimized:** $\log(\text{softmax}(x_i)) = x_i - \log\left(\sum_j e^{x_j}\right)$

This avoids computing the division!

In [ ]:
def log_softmax_naive(x: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """Compute log-softmax naively."""
    return torch.log(torch.softmax(x, dim=dim))

def log_softmax_stable(x: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """Numerically stable log-softmax."""
    x_max = x.max(dim=dim, keepdim=True)[0]
    log_sum_exp = torch.log(torch.exp(x - x_max).sum(dim=dim, keepdim=True))
    return x - x_max - log_sum_exp

# Compare implementations
x = torch.randn(1000, 512, device=device)

# Builtin (optimized)
builtin = F.log_softmax(x, dim=-1)

# Manual stable version
manual = log_softmax_stable(x)

# Naive version
naive = log_softmax_naive(x)

print("Log-Softmax Comparison:")
print(f"Builtin vs Manual: {torch.allclose(builtin, manual, rtol=1e-5)}")
print(f"Builtin vs Naive:  {torch.allclose(builtin, naive, rtol=1e-5)}")

# Benchmark
for name, func in [("Builtin", F.log_softmax), 
                   ("Manual", log_softmax_stable),
                   ("Naive", log_softmax_naive)]:
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(100):
        if name == "Builtin":
            _ = func(x, dim=-1)
        else:
            _ = func(x)
    torch.cuda.synchronize()
    avg_time = ((time.time() - start) / 100) * 1000
    print(f"{name}: {avg_time:.4f} ms")

## Part 4: Combining Operations - Matrix Mult + Softmax

This combination appears in attention mechanisms:

```python
scores = Q @ K.T  # Matrix multiplication
attn = softmax(scores / sqrt(d_k))  # Scaled softmax
```

In [ ]:
def attention_scores(Q: torch.Tensor, K: torch.Tensor, scale: float = None) -> torch.Tensor:
    """Compute attention scores: softmax(Q @ K.T / sqrt(d_k))."""
    d_k = Q.shape[-1]
    if scale is None:
        scale = d_k ** 0.5
    
    # Q: (batch, seq_q, d_k)
    # K: (batch, seq_k, d_k)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / scale
    return torch.softmax(scores, dim=-1)

# Test with transformer dimensions
batch_size = 32
seq_len = 128
d_k = 64

Q = torch.randn(batch_size, seq_len, d_k, device=device)
K = torch.randn(batch_size, seq_len, d_k, device=device)

attn_weights = attention_scores(Q, K)

print(f"Input Q shape: {Q.shape}")
print(f"Input K shape: {K.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\nAttention properties:")
print(f"  Sum per row: {attn_weights.sum(dim=-1)[0, 0]:.6f} (should be 1.0)")
print(f"  Max value: {attn_weights.max():.6f}")
print(f"  Min value: {attn_weights.min():.6f}")

# Visualize attention pattern for first sample
plt.figure(figsize=(10, 8))
plt.imshow(attn_weights[0].cpu().numpy(), cmap='viridis', aspect='auto')
plt.colorbar(label='Attention Weight')
plt.xlabel('Key Position', fontsize=12)
plt.ylabel('Query Position', fontsize=12)
plt.title('Attention Weight Matrix', fontsize=14, fontweight='bold')
plt.show()

### Performance Optimization: Fused Operations

Combining operations reduces memory transfers:

In [ ]:
def benchmark_fused_vs_separate(sizes: list) -> dict:
    """Compare fused vs separate operations."""
    results = {'separate': [], 'fused': []}
    
    for size in sizes:
        Q = torch.randn(32, size, 64, device=device)
        K = torch.randn(32, size, 64, device=device)
        scale = 8.0
        
        # Separate operations
        torch.cuda.synchronize()
        start = time.time()
        scores = torch.matmul(Q, K.transpose(-2, -1))
        scaled = scores / scale
        attn = torch.softmax(scaled, dim=-1)
        torch.cuda.synchronize()
        results['separate'].append((time.time() - start) * 1000)
        
        # Fused operations (single expression)
        torch.cuda.synchronize()
        start = time.time()
        attn = torch.softmax(torch.matmul(Q, K.transpose(-2, -1)) / scale, dim=-1)
        torch.cuda.synchronize()
        results['fused'].append((time.time() - start) * 1000)
    
    return results

seq_sizes = [64, 128, 256, 512]
fusion_results = benchmark_fused_vs_separate(seq_sizes)

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(seq_sizes, fusion_results['separate'], 'o-', label='Separate ops', linewidth=2)
plt.plot(seq_sizes, fusion_results['fused'], 's-', label='Fused ops', linewidth=2)
plt.xlabel('Sequence Length', fontsize=12)
plt.ylabel('Time (ms)', fontsize=12)
plt.title('Fused vs Separate Operations', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

speedups = [sep / fus for sep, fus in zip(fusion_results['separate'], fusion_results['fused'])]
print(f"\nAverage speedup from fusion: {np.mean(speedups):.2f}x")

## Exercise Section

### Exercise 1: Masked Softmax
Implement softmax with masking (used in causal attention):
- Apply mask before softmax to prevent attending to future positions
- Set masked positions to -infinity before softmax

In [ ]:
def masked_softmax(x: torch.Tensor, mask: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """
    Apply softmax with masking.
    Args:
        x: Input tensor
        mask: Boolean mask (True = keep, False = mask)
        dim: Dimension to apply softmax
    """
    # TODO: Implement masked softmax
    pass

# Test with causal mask
# seq_len = 5
# x = torch.randn(1, seq_len, seq_len)
# causal_mask = torch.tril(torch.ones(seq_len, seq_len, dtype=torch.bool))
# result = masked_softmax(x, causal_mask)

### Exercise 2: Batch Matrix Operations
Implement efficient batched matrix multiplication with softmax:
1. Handle 4D tensors (batch, heads, seq, dim)
2. Apply softmax over the correct dimension
3. Benchmark against loops

In [ ]:
# TODO: Implement batched operations
# Hint: Use einsum or batched matrix multiplication

### Exercise 3: Numerical Stability Analysis
Create test cases that demonstrate numerical instability:
1. Generate inputs that cause overflow in naive softmax
2. Show stable version handles them correctly
3. Plot the error as input magnitude increases

In [ ]:
# TODO: Analyze numerical stability

## Summary

### Key Takeaways

✅ **Optimized Matrix Multiplication:**
- Tiling and shared memory reduce global memory access
- Memory coalescing is critical for performance
- PyTorch handles optimization internally

✅ **Stable Softmax:**
- Always subtract max before exponential
- Essential for numerical stability in deep learning
- Log-softmax avoids unnecessary divisions

✅ **Performance Optimization:**
- Fuse operations when possible
- Batch operations for better GPU utilization
- Profile to identify bottlenecks

### Next Steps

In **Notebook 03**, we'll implement:
- Attention mechanisms from scratch (CPU)
- Query, Key, Value concepts
- Scaled dot-product attention
- Visualizing attention patterns

## Further reading (Archive.org)

To better understand GPU-accelerated linear algebra and softmax, search Archive.org for:

- "numerical linear algebra GPU"
- "matrix multiplication CUDA"
- "parallel algorithms softmax"

Focus on materials that discuss tiling, shared memory usage, and reduction operations, since these patterns underlie efficient matrix multiplication and stable softmax implementations on modern GPUs.